In [458]:
def duplicate_and_transform_graph(graph, T):
    l = len(graph)
    new_graph = {}
    offset = len(graph.keys())
    for k, (formula, (mod, neighbours)) in graph.iteritems():
        new_graph[k] = formula, (mod, neighbours)
        new_graph[k + offset] = T + formula, (mod, neighbours)  
    return new_graph

In [365]:
from copy import copy
def clone_graph(graph):
    return {k: (f, (m, copy(ns))) for k, (f, (m, ns)) in graph.iteritems()}

In [ ]:
def apply_prefix_rules(graph, rules):
    new_graph = []
    for formula, (mod, neighbours) in graph:
        for (prefix, replacement) in rules:
            if formula.startswith(rule):
                new_formula = replacement + formula[len(prefix)+1:]
            else:
                new_formula = formula
            new_graph.append((new_formula, (mod, neighbours)))

In [ ]:
def intersection_with_indices(l1, l2):
    common_members = set(l1).intersection(l2)
    return [(x, l1.index(x), l2.index(x)) for x in common_members]

In [ ]:
def pairwise_induction(pairs):
    for (a, b) in pairs:
        if a is not None and b is not None:
            yield (a, b)
        elif a is None:
            yield (b, b)
        else:
            yield (a, a)
            

In [ ]:
list(pairwise_induction([(None,1), (3, 4), (7, None)]))

In [ ]:
def pad(l, n, padding_direction=1, v=None):
    from itertools import repeat
    if len(l) > l:
        return l
    padding = list(repeat(v, n - len(l)))
    if padding_direction > 0:
        return l + padding
    else:
        return padding + l

In [213]:
def has_repeating_values(l):
    without_nones = filter(lambda x: x is not None, l)
    return len(l) > len(set(l))

In [214]:
def padding_zip(l1, l2, padding_direction=1):
    length = max(len(l1), len(l2))
    return zip(pad(l1, length, padding_direction),
               pad(l2, length, padding_direction))

In [347]:
def common_cycle(s1, s2):
    from itertools import repeat, chain
    (c1, l1) = s1
    (c2, l2) = s2
    if c1 and c2 and len(l1) != len(l2):
        raise Exception('Cannot combine closed cycles of different lengths')
    if has_repeating_values(l1):
        raise Exception('Non unique members in cycle 1')
    if has_repeating_values(l2):
        raise Exception('Non unique members in cycle 2')
    tpls = sorted(intersection_with_indices(l1, l2), key= lambda x: x[1])
    if len(tpls) == 0:
        raise Exception('Cannot combine non intersecting sections')
    result = []
    overboard = False
    for i, prv, tpl in zip(range(len(tpls)), tpls, tpls[1:]):
        if tpl[2] < prv[2]:
            ''' List 2 performed a whole cycle '''
            if overboard:
                raise Exception('Ordering is not cyclic')
            overboard = True
            gap = (tpl[1] - prv[1]) - (tpl[2] + len(l2) - prv[2])
            if c2 and gap != 0:
                raise Exception('Mismatch in common index {} (when reaching end of l2, sorted)'.format(i))
            if gap < 0:
                raise Exception('Cannot negative pad in index {} (when reaching end of l2, sorted)'.format(i))
            s1 = [l1[i] for i in range(prv[1], tpl[1])]
            s2 = [l2[i] for i in range(prv[2], len(l2))] + \
                 list(repeat(None, gap)) + \
                 [l2[i] for i in range(0, tpl[2])]
            result = result + zip(s1, s2)
        elif tpl[2] - prv[2] != tpl[1] - prv[1]:
            raise Exception('Mismatch in index {} (sorted)'.format(i))
        else:
            s1 = [l1[i] for i in range(prv[1], tpl[1])]
            s2 = [l2[i] for i in range(prv[2], tpl[2])]
            result = result + zip(s1, s2)
    result_is_closed = c1 or c2 or overboard
    #result = result + \
             
    if result_is_closed:
        ''' List 1 must perform a whole cycle '''
        prv = tpls[-1]
        tpl = tpls[0]
        gap = (tpl[2] - prv[2]) - (tpl[1] + len(l1) - prv[1])
        if c1:
            if gap != 0:
                raise Exception('Mismatch in common index {} (when reaching end of l1, sorted)'.format(i))
        if gap < 0:
            raise Exception('Cannot negative pad in index {} (when reaching end of l1, sorted)'.format(i))
        s2 = [l2[i] for i in range(prv[2], tpl[2])]
        s1 = [l1[i] for i in range(prv[1], len(l1))] + \
             list(repeat(None, gap)) + \
             [l1[i] for i in range(0, tpl[1])]
        result = result + zip(s1, s2)
    else:
        result = padding_zip(l1[0:tpls[0][1]], l2[0:tpls[0][2]], padding_direction=-1) + \
                 result + \
                 padding_zip(l1[tpls[-1][1]:], l2[tpls[-1][2]:])
    return (result_is_closed, list(pairwise_induction(result)))
                

In [259]:
def mirror(tpls):
    (c, l) = tpls
    return (c, [(b, a) for (a, b) in l])

In [278]:
def drop(n, it):
    for i in range(n):
        it.next()
    while True:
        yield it.next()

In [289]:
def compare_cyclic(l1, l2):
    from collections import cycle
    if set(l1) != set(l2) or len(l1) != len(l2):
        return False
    for a,b in zip(l1, drop(l2.index(l1[0]), cycle(l2))):
        if a != b:
            return False
    return True

In [290]:
compare_cyclic([1,2,3], [2,3,1,2,3,1])

False

In [293]:
def test(a, b, expected):
    (c, result) = common_cycle(a, b)
    (cm, mirrored_result) = common_cycle(b, a)
    (cme, mirrored_expected) = mirror(expected)
    if not compare_cyclic(result, expected[1]) or c != expected[0]:
        print '{} and \n{} resulted in \n{} and did not result in \n{}'.format(a, b, result, expected)
    if not compare_cyclic(mirrored_result, mirrored_expected) or cm != cme:
        print '{} and \n{} resulted in \n{} and did not result in \n{}'.format(a, b, mirrored_result, mirror(expected))        

In [294]:
test((False, [0,1]), (False, [1,2]),(False, [(0,0), (1,1), (2, 2)]))
test((False, [0,1,2,3]), (False, [2,3]),(False, [(0,0), (1,1), (2, 2), (3,3)]))
test((False, [0,1,2,3]), (False, [1,2]),(False, [(0,0), (1,1), (2, 2), (3,3)]))
test((False, [0,1,2,3]), (False, [1,None,3]),(False, [(0,0), (1,1), (2, 2), (3,3)]))
test((False, [0,1,2,3]), (False, [2,None,0]),(True, [(0,0), (1,1), (2, 2), (3,3)]))
test((False, [1,2]), (False, [2,3]),(False, [(1,1), (2,2), (3,3)]))

In [12]:
def common_member(l1, l2):    
    for i1 in range(len(l1)):
        x1 = l1[i1]
        if x1 in l2:
            return (i1, l2.index(l1[i1]))
    return (-1, -1)

In [504]:
def index_or_identity(l, i):
    return l[i] if i in l else i

In [387]:
def without(l, indices):
    return [l[i] for i in range(len(l)) if i not in indices]

def without_keys(d, keys):
    return {k: v for k, v in d.iteritems() if k not in keys}

In [431]:
def replace(graph, i1, i2):
    replacer = lambda x: i2 if x == i1 else x
    return {
        k: (f, (c, map(replacer, n))) for k, (f, (c, n)) in graph.iteritems()
    }

def scan_for_errors(graph):
    for k, (f, (c, n)) in graph.iteritems():
        if len(n) != len(set(n)):
            print graph
            raise Exception('Non unique members in node {}'.format(k))

In [506]:
from functools import partial
def merge_vertices(graph, merging_table):
    '''
    vertex of index i1 is merged into i2
    i1 - source index
    i2 - destination index
    '''
    if i1 == i2:
        return graph
    f1, (c1, n1) = graph[i1]
    f2, (c2, n2) = graph[i2]
    mapper = partial(map, partial(index_or_identity_or_identity, merging_table))
    cc = common_cycle((c1, mapper(n1)), (c2, mapper(n2)))
    new_graph = replace(graph, i1, i2)
    new_graph[i2] = (f2, (cc[0], [x[1] for x in cc[1]]))
    new_graph = without_keys(new_graph, {i1})    
    local_merging_table = {}
    for src, dst in cc[1]:
        if src != dst:
            local_merging_table[src] = dst
    for src, dst in local_merging_table.iteritems()
    return new_graph

In [423]:
graph = {
    0: ('P', (False, [1, 2, 3, 4])),
    1: ('a', (False, [0, 2])),
    2: ('b', (False, [1, 0, 3])),
    3: ('c', (False, [2, 0, 4])),    
    4: ('d', (False, [3, 0])),
    9: ('9', (False, [5, 6, 7, 8])),
    5: ('e', (False, [9, 6])),
    6: ('f', (False, [5, 9, 7])),
    7: ('g', (False, [6, 9, 8])),
    8: ('h', (False, [7, 9])),
}


In [502]:
actions = ['a', 'b']
rules = [('ab', ''), 
         ('cd', ''),
         ('aaa', ''),
         ('bbb', ''),
         ('aP', 'P'),
         ('babP', 'P')]


initial_graph = {
    0: ('P', (False, [1, 2])),
    1: ('bP', (False, [2, 0])),
    2: ('bbP', (False, [0, 1]))
}

path = 'aaa'

In [503]:
from collections import defaultdict
graph = initial_graph
for c in path:
    grouped_by_formula = defaultdict(list)
    new_graph = duplicate_and_transform_graph(graph, c)    
    for k, (f, x) in graph.iteritems():
        new_formula = c + f
        for fr, to in rules:
            if new_formula.startswith(fr):
                new_formula = new_formula.replace(fr, to)
        grouped_by_formula[new_formula].append(k)
        new_graph[k] = (new_formula, x)    
    print grouped_by_formula
    for f, group in grouped_by_formula.iteritems():
        if len(group) == 2:
            print new_graph
            print group
            new_graph = merge_vertices(new_graph, group[0], group[1])
            print 'merged', new_graph
        elif len(group) > 2:
            print f, group
            raise Exception('A group larger than 2')
    graph = new_graph

defaultdict(<type 'list'>, {'P': [0, 1], 'bP': [2]})
{0: ('P', (False, [1, 2])), 1: ('P', (False, [2, 0])), 2: ('bP', (False, [0, 1])), 3: ('aP', (False, [1, 2])), 4: ('abP', (False, [2, 0])), 5: ('abbP', (False, [0, 1]))}
[0, 1]
merged {1: ('P', (False, [1, 2, 0])), 2: ('bP', (False, [1, 1])), 3: ('aP', (False, [1, 2])), 4: ('abP', (False, [2, 1])), 5: ('abbP', (False, [1, 1]))}
defaultdict(<type 'list'>, {'aabP': [4], 'P': [1, 2], 'aaP': [3], 'aabbP': [5]})
{1: ('P', (False, [1, 2, 0])), 2: ('P', (False, [1, 1])), 3: ('aaP', (False, [1, 2])), 4: ('aabP', (False, [2, 1])), 5: ('aabbP', (False, [1, 1])), 6: ('aP', (False, [1, 2, 0])), 7: ('abP', (False, [1, 1])), 8: ('aaP', (False, [1, 2])), 9: ('aabP', (False, [2, 1])), 10: ('aabbP', (False, [1, 1]))}
[1, 2]


Exception: Non unique members in cycle 2

In [501]:
graph

{1: ('P', (False, [1, 2, 0])),
 2: ('bP', (False, [1, 1])),
 3: ('aP', (False, [1, 2])),
 4: ('abP', (False, [2, 1])),
 5: ('abbP', (False, [1, 1]))}